In [12]:
# %pip install confz
# %pip install -r Hack_2024/requirements.txt

In [ ]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import cv2
import numpy as np
from PIL import Image
from PIL.ExifTags import TAGS
from datetime import datetime, timedelta
import os
import csv

# Загрузка модели
def load_model(model_path):
    model = torch.load(model_path, map_location=torch.device('cpu'))
    model.eval()
    return model

# Предобработка изображения
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    return torchvision.transforms.ToTensor()(image).unsqueeze(0)

# Получение временной метки фотографии
def get_photo_timestamp(file_path):
    try:
        with Image.open(file_path) as img:
            exif = img._getexif()
            if exif:
                for tag_id, value in exif.items():
                    tag = TAGS.get(tag_id, tag_id)
                    if tag == 'DateTimeOriginal':
                        return datetime.strptime(value, '%Y:%m:%d %H:%M:%S')
                
                for tag_id, value in exif.items():
                    tag = TAGS.get(tag_id, tag_id)
                    if tag in ['DateTime', 'DateTimeDigitized']:
                        return datetime.strptime(value, '%Y:%m:%d %H:%M:%S')
    except Exception as e:
        print(f"Ошибка при чтении EXIF данных: {e}")
    
    return datetime.fromtimestamp(os.path.getmtime(file_path))

# Определение новой сцены
def is_new_scene(current_time, last_photo_time):
    if last_photo_time is None:
        return True
    time_difference = current_time - last_photo_time
    return time_difference > timedelta(minutes=30)

# Вычисление IoU
def calculate_iou(box1, box2):
    x1, y1, x2, y2 = box1
    x3, y3, x4, y4 = box2
    
    xx1 = max(x1, x3)
    yy1 = max(y1, y3)
    xx2 = min(x2, x4)
    yy2 = min(y2, y4)
    
    intersection = max(0, xx2 - xx1) * max(0, yy2 - yy1)
    
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x4 - x3) * (y4 - y3)
    
    iou = intersection / float(box1_area + box2_area - intersection)
    return iou

# Обработка изображения с помощью YOLO
def process_image_with_yolo(model, image_path, labels):
    image = preprocess_image(image_path)
    with torch.no_grad():
        predictions = model(image)
    
    boxes = predictions[0]['boxes'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()
    
    confidence_threshold = 0.3
    filtered_indices = scores > confidence_threshold
    
    filtered_boxes = boxes[filtered_indices]
    filtered_scores = scores[filtered_indices]
    filtered_labels = labels[filtered_indices]
    
    # Сортировка предсказаний по уверенности (от большей к меньшей)
    sorted_indices = np.argsort(filtered_scores)[::-1]
    filtered_boxes = filtered_boxes[sorted_indices]
    filtered_scores = filtered_scores[sorted_indices]
    filtered_labels = filtered_labels[sorted_indices]
    
    results = []
    used_detections = set()
    
    for i in range(len(filtered_boxes)):
        if i in used_detections:
            continue
        
        bbox = filtered_boxes[i]
        score = filtered_scores[i]
        label = filtered_labels[i]
        
        current_bbox = bbox.tolist()
        
        # Проверка перекрытия с другими боксами
        for j in range(i + 1, len(filtered_boxes)):
            if j not in used_detections:
                other_bbox = filtered_boxes[j].tolist()
                iou = calculate_iou(current_bbox, other_bbox)
                if iou > 0.3:  # Порог IoU
                    used_detections.add(j)
        
        results.append({
            'class': labels[label],
            'confidence': float(score),
            'bbox': current_bbox
        })
    
    return results

# Основная функция обработки папки с изображениями
def process_folder(folder_path, model, labels):
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
    image_files.sort(key=lambda x: get_photo_timestamp(os.path.join(folder_path, x)))
    
    scenes = []
    current_scene = None
    last_photo_time = None
    last_class = None
    last_detection_time = None
    
    for image_file in image_files:
        file_path = os.path.join(folder_path, image_file)
        timestamp = get_photo_timestamp(file_path)
        
        if current_scene is None or is_new_scene(timestamp, last_photo_time):
            if current_scene is not None:
                scenes.append(current_scene)
            current_scene = {
                'photos': [],
                'animal_ids': set(),
                'max_unique_animals': 0,
                'bboxes': []
            }
        
        results = process_image_with_yolo(model, file_path, labels)
        
        bbox_strings = []
        current_photo_animal_ids = set()
        for detection in results:
            x1, y1, x2, y2 = map(int, detection['bbox'])
            category = detection['class']
            
            # Фильтрация выбросов
            if last_class is not None and (timestamp - last_detection_time) < timedelta(seconds=30):
                category = last_class
            
            animal_id = f"animal_{len(current_scene['animal_ids']) + 1}"
            current_scene['animal_ids'].add(animal_id)
            
            bbox_strings.append(f"{x1},{y1},{x2},{y2},{animal_id},{category}")
            current_photo_animal_ids.add(animal_id)
            current_scene['bboxes'].append((x1, y1, x2, y2))
            
            last_class = category
            last_detection_time = timestamp
        
        bbox_string = ";".join(bbox_strings)
        
        animal_count = len(bbox_strings)
        unique_animal_count = len(current_photo_animal_ids)
        current_scene['max_unique_animals'] = max(current_scene['max_unique_animals'], unique_animal_count)
        
        current_scene['photos'].append({
            'path': file_path,
            'timestamp': timestamp,
            'animal_count': animal_count,
            'unique_animal_count': unique_animal_count,
            'bbox_string': bbox_string
        })
        
        last_photo_time = timestamp
    
    if current_scene is not None:
        scenes.append(current_scene)
    
    return scenes

# Функция для экспорта данных в CSV
def export_to_csv(scenes, output_path):
    with open(output_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['folder_name', 'class', 'date_registration_start', 'date_registration_end', 'count'])
        
        for scene in scenes:
            folder_name = os.path.dirname(scene['photos'][0]['path'])
            start_time = scene['photos'][0]['timestamp']
            end_time = scene['photos'][-1]['timestamp']
            
            class_counts = {}
            for photo in scene['photos']:
                bboxes = photo['bbox_string'].split(';')
                for bbox in bboxes:
                    _, _, _, _, _, class_name = bbox.split(',')
                    class_counts[class_name] = class_counts.get(class_name, 0) + 1
            
            for class_name, count in class_counts.items():
                writer.writerow([
                    os.path.basename(folder_name),
                    class_name,
                    start_time.strftime('%Y-%m-%d %H:%M:%S'),
                    end_time.strftime('%Y-%m-%d %H:%M:%S'),
                    count
                ])

# Пример использования
model_path = 'runs/detect/train11/weights/best.pt'
labels_path = 'labels.txt'

# Загрузка модели и меток
model = load_model(model_path)
with open(labels_path, 'r') as f:
    labels = [line.strip() for line in f.readlines()]

In [ ]:
for folder_path in glob('test_data_Minprirodi/*'):
    
    folder = folder_path.split('/')[-1]
    output_csv_path = f'output/results_{folder}.csv'
    # Обработка папки с изображениями
    scenes = process_folder(folder_path, model, labels)

    # Экспорт результатов в CSV
    export_to_csv(scenes, output_csv_path)

    print(f"Результаты экспортированы в {output_csv_path} для {folder}")

In [69]:
import csv
from datetime import datetime, timedelta
import os
from PIL import Image
from PIL.ExifTags import TAGS

def get_photo_timestamp(file_path):
    try:
        with Image.open(file_path) as img:
            exif = img._getexif()
            if exif:
                for tag_id, value in exif.items():
                    tag = TAGS.get(tag_id, tag_id)
                    if tag == 'DateTimeOriginal':
                        return datetime.strptime(value, '%Y:%m:%d %H:%M:%S')
                
                for tag_id, value in exif.items():
                    tag = TAGS.get(tag_id, tag_id)
                    if tag in ['DateTime', 'DateTimeDigitized']:
                        return datetime.strptime(value, '%Y:%m:%d %H:%M:%S')
    except Exception as e:
        print(f"Ошибка при чтении EXIF данных: {e}")
    
    return datetime.fromtimestamp(os.path.getmtime(file_path))

def is_new_scene(current_time, last_photo_time):
    if last_photo_time is None:
        return True
    time_difference = current_time - last_photo_time
    return time_difference > timedelta(minutes=40)

def process_csv_data(input_csv_path, output_csv_path, images_folder):
    scenes = []
    current_scene = None
    last_photo_time = None

    with open(input_csv_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            image_name = row['image_name']
            image_path = os.path.join(images_folder, image_name)
            timestamp = get_photo_timestamp(image_path)
            
            if current_scene is None or is_new_scene(timestamp, last_photo_time):
                if current_scene is not None:
                    scenes.append(current_scene)
                current_scene = {
                    'photos': [],
                    'animal_ids': set(),
                    'max_unique_animals': 0,
                    'class_counts': {}
                }
            
            class_name = row['class_name']
            count = int(row['count'])
            confidence = float(row['confidence'])
            
            current_scene['photos'].append({
                'image_name': image_name,
                'timestamp': timestamp,
                'class_name': class_name,
                'count': count,
                'confidence': confidence
            })
            
            current_scene['class_counts'][class_name] = current_scene['class_counts'].get(class_name, 0) + count
            current_scene['max_unique_animals'] = max(current_scene['max_unique_animals'], count)
            
            last_photo_time = timestamp
    
    if current_scene is not None:
        scenes.append(current_scene)
    
    # Экспорт результатов в новый CSV файл
    with open(output_csv_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['folder_name', 'class', 'date_registration_start', 'date_registration_end', 'count'])
        
        for scene in scenes:
            folder_name = os.path.dirname(scene['photos'][0]['image_name'])
            start_time = scene['photos'][0]['timestamp']
            end_time = scene['photos'][-1]['timestamp']
            
            for class_name, count in scene['class_counts'].items():
                writer.writerow([
                    folder_name,
                    class_name,
                    start_time.strftime('%Y-%m-%d %H:%M:%S'),
                    end_time.strftime('%Y-%m-%d %H:%M:%S'),
                    count
                ])
    
    return scenes

In [70]:
!pwd

/home/jupyter/work/resources


In [71]:
import os
if os.path.exists('table_final.csv'):
    os.remove('table_final.csv')

!python3 Hack_2024/main.py

# Пример использования
input_csv_path = 'table_final.csv'
output_csv_path = '55.csv'
images_folder = 'test_data_Minprirodi/traps/55'  # Укажите путь к папке с изображениями

scenes = process_csv_data(input_csv_path, output_csv_path, images_folder)

import pandas as pd
df = pd.read_csv('55.csv')
df['folder_name'] = 55
df.to_csv('55.csv', index=False)

1246


100%|██████████| 1246/1246 [02:22<00:00,  8.76it/s]


In [72]:
folder = 51

import os
if os.path.exists('table_final.csv'):
    os.remove('table_final.csv')
    
with open('Hack_2024/configs/config.yml', 'r') as f:
    text = f.read()
text = text.replace('55', f'{folder}')
with open('Hack_2024/configs/config.yml', 'w') as f:
    f.write(text)

!python3 Hack_2024/main.py

# Пример использования
input_csv_path = 'table_final.csv'
output_csv_path = f'{folder}.csv'
images_folder = f'test_data_Minprirodi/traps/{folder}'  # Укажите путь к папке с изображениями

scenes = process_csv_data(input_csv_path, output_csv_path, images_folder)

import pandas as pd
df = pd.read_csv(f'{folder}.csv')
df['folder_name'] = folder
df.to_csv(f'{folder}.csv', index=False)

423


100%|██████████| 423/423 [00:35<00:00, 11.99it/s]


In [73]:
folder = 52

import os
if os.path.exists('table_final.csv'):
    os.remove('table_final.csv')
    
with open('Hack_2024/configs/config.yml', 'r') as f:
    text = f.read()
text = text.replace('51', f'{folder}')
with open('Hack_2024/configs/config.yml', 'w') as f:
    f.write(text)

!python3 Hack_2024/main.py

# Пример использования
input_csv_path = 'table_final.csv'
output_csv_path = f'{folder}.csv'
images_folder = f'test_data_Minprirodi/traps/{folder}'  # Укажите путь к папке с изображениями

scenes = process_csv_data(input_csv_path, output_csv_path, images_folder)

import pandas as pd
df = pd.read_csv(f'{folder}.csv')
df['folder_name'] = folder
df.to_csv(f'{folder}.csv', index=False)

564


100%|██████████| 564/564 [00:48<00:00, 11.71it/s]


In [74]:
folder = 53

import os
if os.path.exists('table_final.csv'):
    os.remove('table_final.csv')
    
with open('Hack_2024/configs/config.yml', 'r') as f:
    text = f.read()
text = text.replace('52', f'{folder}')
with open('Hack_2024/configs/config.yml', 'w') as f:
    f.write(text)

!python3 Hack_2024/main.py

# Пример использования
input_csv_path = 'table_final.csv'
output_csv_path = f'{folder}.csv'
images_folder = f'test_data_Minprirodi/traps/{folder}'  # Укажите путь к папке с изображениями

scenes = process_csv_data(input_csv_path, output_csv_path, images_folder)

import pandas as pd
df = pd.read_csv(f'{folder}.csv')
df['folder_name'] = folder
df.to_csv(f'{folder}.csv', index=False)

231


100%|██████████| 231/231 [00:20<00:00, 11.47it/s]


In [75]:
folder = 54

import os
if os.path.exists('table_final.csv'):
    os.remove('table_final.csv')
    
with open('Hack_2024/configs/config.yml', 'r') as f:
    text = f.read()
text = text.replace('53', f'{folder}')
with open('Hack_2024/configs/config.yml', 'w') as f:
    f.write(text)

!python3 Hack_2024/main.py

# Пример использования
input_csv_path = 'table_final.csv'
output_csv_path = f'{folder}.csv'
images_folder = f'test_data_Minprirodi/traps/{folder}'  # Укажите путь к папке с изображениями

scenes = process_csv_data(input_csv_path, output_csv_path, images_folder)

import pandas as pd
df = pd.read_csv(f'{folder}.csv')
df['folder_name'] = folder
df.to_csv(f'{folder}.csv', index=False)

832


100%|██████████| 832/832 [01:36<00:00,  8.58it/s]


In [76]:
folder = 56

import os
if os.path.exists('table_final.csv'):
    os.remove('table_final.csv')
    
with open('Hack_2024/configs/config.yml', 'r') as f:
    text = f.read()
text = text.replace('54', f'{folder}')
with open('Hack_2024/configs/config.yml', 'w') as f:
    f.write(text)

!python3 Hack_2024/main.py

# Пример использования
input_csv_path = 'table_final.csv'
output_csv_path = f'{folder}.csv'
images_folder = f'test_data_Minprirodi/traps/{folder}'  # Укажите путь к папке с изображениями

scenes = process_csv_data(input_csv_path, output_csv_path, images_folder)

import pandas as pd
df = pd.read_csv(f'{folder}.csv')
df['folder_name'] = folder
df.to_csv(f'{folder}.csv', index=False)

140


100%|██████████| 140/140 [00:17<00:00,  7.89it/s]


In [77]:
folder = 57

import os
if os.path.exists('table_final.csv'):
    os.remove('table_final.csv')
    
with open('Hack_2024/configs/config.yml', 'r') as f:
    text = f.read()
text = text.replace('56', f'{folder}')
with open('Hack_2024/configs/config.yml', 'w') as f:
    f.write(text)

!python3 Hack_2024/main.py

# Пример использования
input_csv_path = 'table_final.csv'
output_csv_path = f'{folder}.csv'
images_folder = f'test_data_Minprirodi/traps/{folder}'  # Укажите путь к папке с изображениями

scenes = process_csv_data(input_csv_path, output_csv_path, images_folder)

import pandas as pd
df = pd.read_csv(f'{folder}.csv')
df['folder_name'] = folder
df.to_csv(f'{folder}.csv', index=False)

399


100%|██████████| 399/399 [00:34<00:00, 11.47it/s]


In [78]:
folder = 58

import os
if os.path.exists('table_final.csv'):
    os.remove('table_final.csv')
    
with open('Hack_2024/configs/config.yml', 'r') as f:
    text = f.read()
text = text.replace('57', f'{folder}')
with open('Hack_2024/configs/config.yml', 'w') as f:
    f.write(text)

!python3 Hack_2024/main.py

# Пример использования
input_csv_path = 'table_final.csv'
output_csv_path = f'{folder}.csv'
images_folder = f'test_data_Minprirodi/traps/{folder}'  # Укажите путь к папке с изображениями

scenes = process_csv_data(input_csv_path, output_csv_path, images_folder)

import pandas as pd
df = pd.read_csv(f'{folder}.csv')
df['folder_name'] = folder
df.to_csv(f'{folder}.csv', index=False)

645


100%|██████████| 645/645 [01:46<00:00,  6.03it/s]


In [79]:
folder = 59

import os
if os.path.exists('table_final.csv'):
    os.remove('table_final.csv')
    
with open('Hack_2024/configs/config.yml', 'r') as f:
    text = f.read()
text = text.replace('58', f'{folder}')
with open('Hack_2024/configs/config.yml', 'w') as f:
    f.write(text)

!python3 Hack_2024/main.py

# Пример использования
input_csv_path = 'table_final.csv'
output_csv_path = f'{folder}.csv'
images_folder = f'test_data_Minprirodi/traps/{folder}'  # Укажите путь к папке с изображениями

scenes = process_csv_data(input_csv_path, output_csv_path, images_folder)

import pandas as pd
df = pd.read_csv(f'{folder}.csv')
df['folder_name'] = folder
df.to_csv(f'{folder}.csv', index=False)

507


100%|██████████| 507/507 [00:43<00:00, 11.57it/s]


In [80]:
folder = 60

import os
if os.path.exists('table_final.csv'):
    os.remove('table_final.csv')
    
with open('Hack_2024/configs/config.yml', 'r') as f:
    text = f.read()
text = text.replace('59', f'{folder}')
with open('Hack_2024/configs/config.yml', 'w') as f:
    f.write(text)

!python3 Hack_2024/main.py

# Пример использования
input_csv_path = 'table_final.csv'
output_csv_path = f'{folder}.csv'
images_folder = f'test_data_Minprirodi/traps/{folder}'  # Укажите путь к папке с изображениями

scenes = process_csv_data(input_csv_path, output_csv_path, images_folder)

import pandas as pd
df = pd.read_csv(f'{folder}.csv')
df['folder_name'] = folder
df.to_csv(f'{folder}.csv', index=False)

1355


100%|██████████| 1355/1355 [02:36<00:00,  8.64it/s]


In [81]:
res = pd.concat([
    pd.read_csv('51.csv'),
    pd.read_csv('52.csv'),
    pd.read_csv('53.csv'),
    pd.read_csv('54.csv'),
    pd.read_csv('55.csv'),
    pd.read_csv('56.csv'),
    pd.read_csv('57.csv'),
    pd.read_csv('58.csv'),
    pd.read_csv('59.csv'),
    pd.read_csv('60.csv'),
])

In [82]:
res = res.rename(columns={'folder_name': 'name_folder'})

In [83]:
res.to_csv('res2.csv', index=False)

In [84]:
res

,name_folder,class,date_registration_start,date_registration_end,count
0,51,Bear,2018-11-19 19:34:30,2018-11-19 19:34:34,6
1,51,Bear,2018-11-29 12:03:56,2018-11-29 12:04:00,4
2,51,Tiger,2018-11-30 14:47:57,2018-11-30 14:47:58,3
3,51,Lynx,2018-12-21 17:16:22,2018-12-21 17:16:24,3
4,51,Roe_Deer,2019-01-04 10:09:40,2019-01-04 10:10:27,9
...,...,...,...,...,...
245,60,Roe_Deer,2019-10-16 19:55:25,2019-10-16 19:55:56,4
246,60,Musk_Deer,2019-10-16 19:55:25,2019-10-16 19:55:56,3
247,60,Bear,2019-10-20 00:31:43,2019-10-20 00:31:45,1
248,60,Musk_Deer,2019-10-20 00:31:43,2019-10-20 00:31:45,2


In [17]:
# !unzip -P FnmZz4qRHDfxE9GV25rXU8 test_dataset_test_data_Minprirodi.zip 

In [5]:
# !unzip yolo_data.zip

In [6]:
# %pip install ultralytics

In [12]:
!pwd

/home/jupyter/work/resources


In [21]:
from ultralytics import YOLO

# Load YOLOv10n model from scratch
model = YOLO("yolov10s.pt")

# Train the model
model.train(data="./yolo_data/data.yml", epochs=100, imgsz=1024, batch=16)

100%|██████████| 15.9M/15.9M [00:00<00:00, 24.8MB/s]


Ultralytics YOLOv8.2.50 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-PCIE-32GB, 32501MiB)
engine/trainer: task=detect, mode=train, model=yolov10s.pt, data=./yolo_data/data.yml, epochs=100, time=None, patience=100, batch=16, imgsz=1024, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

train: Scanning /home/jupyter/work/resources/yolo_data/train.cache... 17812 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17812/17812 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /home/jupyter/work/resources/yolo_data/val.cache... 1980 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1980/1980 [00:00<?, ?it/s]


...

     41/100      25.5G      1.243      1.319      2.207         10       1024: 100%|██████████| 1114/1114 [05:49<00:00,  3.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:21<00:00,  2.85it/s]

                   all       1980       1980      0.841      0.841      0.901      0.817



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     42/100      25.5G      1.212      1.307       2.19         32       1024:  27%|██▋       | 302/1114 [01:35<04:16,  3.17it/s]


KeyboardInterrupt: 

In [39]:
import os
import cv2
import torch
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from tqdm import tqdm

# Функция для вычисления IoU
def bbox_iou(box1, box2):
    inter_area = (
        max(0, min(box1[2], box2[2]) - max(box1[0], box2[0])) *
        max(0, min(box1[3], box2[3]) - max(box1[1], box2[1]))
    )
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    iou = inter_area / float(box1_area + box2_area - inter_area)
    return iou

# Загрузка модели YOLOv10
model = YOLO('runs/detect/train10/weights/best.pt')

# Путь к исходным данным и к новым данным
src_path = Path('./yolo_data')
dst_path = Path('./yolo_data2')
dst_path.mkdir(parents=True, exist_ok=True)

for phase in ['train', 'val']:
    (dst_path / phase).mkdir(parents=True, exist_ok=True)

    # Список файлов изображений
    image_files = list((src_path / phase).glob('*.jpg'))

    for image_file in tqdm(image_files):
        # Загрузка изображения
        img = cv2.imread(str(image_file))
        height, width = img.shape[:2]

        # Применение модели для предсказаний
        results = model(img, verbose=False)
        new_boxes = []
        for result, conf, cls_ in zip(results[0].boxes.xyxy.cpu().numpy(), results[0].boxes.conf.cpu().numpy(), results[0].boxes.cls.cpu().numpy()):  # xyxy формат
            if conf > 0.7:  # Уверенность > 0.7
                x1, y1, x2, y2 = result
                new_boxes.append([x1, y1, x2, y2, cls_, conf])

        # Чтение старых аннотаций
        label_file = image_file.with_suffix('.txt')
        old_boxes = []
        if label_file.exists():
            with open(label_file, 'r') as f:
                for line in f.readlines():
                    cls, x, y, w, h = map(float, line.strip().split())
                    x1 = (x - w / 2) * width
                    y1 = (y - h / 2) * height
                    x2 = (x + w / 2) * width
                    y2 = (y + h / 2) * height
                    old_boxes.append([x1, y1, x2, y2, cls])

        # Слияние старых и новых аннотаций
        final_boxes = old_boxes[:]
        for new_box in new_boxes:
            keep = True
            for old_box in old_boxes:
                if bbox_iou(new_box[:4], old_box[:4]) > 0.5:
                    keep = False
                    break
            if keep:
                final_boxes.append(new_box)

        # Сохранение изображения и аннотаций в новую директорию
        new_image_file = dst_path / phase / image_file.name
        new_label_file = new_image_file.with_suffix('.txt')
        cv2.imwrite(str(new_image_file), img)
        with open(new_label_file, 'w') as f:
            for box in final_boxes:
                x_center = (box[0] + box[2]) / 2 / width
                y_center = (box[1] + box[3]) / 2 / height
                w = (box[2] - box[0]) / width
                h = (box[3] - box[1]) / height
                cls = int(box[4])
                f.write(f"{cls} {x_center} {y_center} {w} {h}\n")


100%|██████████| 1980/1980 [04:28<00:00,  7.36it/s]


In [40]:
from ultralytics import YOLO

# Load YOLOv10n model from scratch
model = YOLO("runs/detect/train10/weights/best.pt")

# Train the model
model.train(data="./yolo_data2/data.yml", epochs=50, imgsz=1024, batch=16)

Ultralytics YOLOv8.2.50 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-PCIE-32GB, 32501MiB)
engine/trainer: task=detect, mode=train, model=runs/detect/train10/weights/best.pt, data=./yolo_data2/data.yml, epochs=50, time=None, patience=100, batch=16, imgsz=1024, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show

train: Scanning /home/jupyter/work/resources/yolo_data2/train... 17812 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17812/17812 [00:21<00:00, 831.65it/s]


train: New cache created: /home/jupyter/work/resources/yolo_data2/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /home/jupyter/work/resources/yolo_data2/val... 1980 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1980/1980 [00:04<00:00, 425.77it/s]


val: New cache created: /home/jupyter/work/resources/yolo_data2/val.cache
Plotting labels to runs/detect/train11/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 99 weight(decay=0.0), 112 weight(decay=0.0005), 111 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1024 train, 1024 val
Using 8 dataloader workers
Logging results to runs/detect/train11
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      15.3G     0.6375     0.3866      1.823          4       1024: 100%|██████████| 1114/1114 [05:48<00:00,  3.20it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:22<00:00,  2.75it/s]


                   all       1980       2060       0.93      0.878      0.956      0.891

50 epochs completed in 5.293 hours.
Optimizer stripped from runs/detect/train11/weights/last.pt, 16.6MB
Optimizer stripped from runs/detect/train11/weights/best.pt, 16.6MB

Validating runs/detect/train11/weights/best.pt...
Ultralytics YOLOv8.2.50 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-PCIE-32GB, 32501MiB)
YOLOv10s summary (fused): 293 layers, 8049666 parameters, 0 gradients, 24.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:23<00:00,  2.61it/s]


                   all       1980       2060       0.93      0.878      0.956      0.891
                Badger          5          5          1      0.837      0.995      0.879
                  Bear        122        122      0.972      0.959      0.992      0.962
             Wolverine          2          2      0.886          1      0.995      0.921
                Marten         11         11      0.877       0.65      0.841      0.747
              Roe_Deer        682        711      0.955      0.921      0.979      0.932
              Red_Deer        183        190      0.916      0.914      0.969      0.937
                 Tiger         22         22      0.994          1      0.995      0.954
              Squirrel          3          3      0.916      0.667      0.863       0.77
                   Fox         59         64      0.963      0.811      0.947      0.832
          Snow_Leopard          2          2      0.901          1      0.995      0.949
                  Lyn

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fe195b0d000>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,

In [41]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("runs/detect/train11/weights/best.pt")

# Export the model to ONNX format
model.export(format="onnx")  # creates 'yolov8n.onnx'

Ultralytics YOLOv8.2.50 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Core Processor (Broadwell, no TSX))
YOLOv10s summary (fused): 293 layers, 8049666 parameters, 0 gradients, 24.5 GFLOPs

PyTorch: starting from 'runs/detect/train11/weights/best.pt' with input shape (1, 3, 1024, 1024) BCHW and output shape(s) (1, 300, 6) (15.9 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 77.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 26.5s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.16.1 opset 17...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: export success ✅ 30.2s, saved as 'runs/detect/train11/weights/best.onnx' (28.0 MB)

Export complete (34.6s)
Results saved to /home/jupyter/work/resources/runs/detect/train11/weights
Predict:         yolo predict task=detect model=runs/detect/train11/weights/best.onnx imgsz=1024  
Validate:        yolo val task=detect model=runs/detect/train11/weights/best.onnx imgsz=1024 data=./yolo_data2/data.yml  
Visualize:       http

'runs/detect/train11/weights/best.onnx'